In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Monte Carlo

Monte Carlo simulation rely on repeated random sampling from known or assumed
probability distributions. 

Example

We are throwing darts into a square which sizes of lenght 2. The dart will fall
in the square, necessarily, but its exactly position is random. There is a circle
of radius 1 inside. WHats is the probability that the darts fall inside of the 
circle?

We now that the probability oof throwing in the circle is proportional to the 
area of the circle ($\pi/4$). BUt let's solve this using monte carlo.

<img src="images/monte_carlo_darts.png" alt="Center" width="450"/>



In [4]:

n = 10**6    # number of simulations
inside_circle = 0   # how many darts fell inside the circle (we care about circ/n)

for i in range(n):
    x = np.random.uniform(-1, 1) # horizontal point
    y = np.random.uniform(-1, 1) # vertical point

    # Does the point fall inside the circle?

    if x**2 + y**2 <=1:
        inside_circle += +1

print(f"inside_circle/total: {inside_circle/n}")

inside_circle/total: 0.785833


# Markov Chains

Markov chain is a mathematical model that represents a system that  transitions 
from one state to another within a finite or countable number of possible states. 
It is a stochastic process characterized by the **Markov property**, which states 
that the future state depends only on the current state and not on the sequence 
of events that preceded it.

Formally,

$$
P(X_{t+1} = x | X_n = x_t, X_{t-1} = x_{t-1}, \dots, X_0 = x_0) = P(X_{t+1} = x | X_t = x_t)
$$

* State Space: The set of all possible states.
* Initial State Distribution: Probabilities of starting states.
* Transition Probabilities: Probabilities of moving from one state to another.
* Transition Matrix: A square matrix where each element  $P_{ij}$ represents the transition probability from state $i$ to state  $j$.


### Example: Web Page Navigation

Suppose we’re modeling a user’s navigation between web pages on a website. The 
pages are:1-Home, 2-About, 3-Products, 4-Contact. The end goal of this exercise 
is to simulate and analyze the long-term behavior of a user’s navigation patterns 
to answer questions such as:
1. What is the likelihood that a user will visit each page over time?
2. How does the user’s navigation evolve over multiple steps?
3. What is the expected frequency of visits to each page? Which one is the most
popular?
4. Does the user tend to get “stuck” on certain pages (absorbing states)?


**Transition Matrix**
| From \ To | Home | About | Products | Contact |
|-----------|------|-------|----------|---------|
| Home      | 0.1  | 0.4   | 0.4      | 0.1     |
| About     | 0.2  | 0.1   | 0.6      | 0.1     |
| Products  | 0.3  | 0.2   | 0.4      | 0.1     |
| Contact   | 0.5  | 0.2   | 0.2      | 0.1     |

Moreover, we are assuming that all user starts from the home page (this defines 
the initial state probabilities). 

In [3]:
# Setting states and transition matrix

states = ['Home', 'About', 'Products', 'Contact']

transition_matrix = np.array([
    [0.1, 0.4, 0.4, 0.1],  # From Home
    [0.2, 0.1, 0.6, 0.1],  # From About
    [0.3, 0.2, 0.4, 0.1],  # From Products
    [0.5, 0.2, 0.2, 0.1],  # From Contact
])

In [11]:
type(transition_matrix)

numpy.ndarray

In [4]:
initial_state = 0
transition_matrix[initial_state]

array([0.1, 0.4, 0.4, 0.1])

In [21]:
def markov_chain(transition_matrix, states, initial_state=0, num_steps=100):
    """"    
    Returns:
    -------
    state_sequence : list
        Sequence of states visited during the simulation.
    state_indices : list
        Sequence of state indices corresponding to the states.
    """

    # Initialization
    current_state = initial_state
    state_sequence= [states[current_state]]
    state_indices = [current_state]

    for _ in range(num_steps):
        # Get transition probabilities for current state
        transition_probs = transition_matrix[current_state] 

        # Choose the next state based on transition probabilities
        next_state = np.random.choice(
            range(len(states)),  # Choose the index (integer)
            p = transition_probs )
        
        # Update current state
        current_state = next_state

        #Record the state
        state_sequence.append(states[current_state])
        state_indices.append(current_state)

    return state_sequence, state_indices

In [23]:

state_sequence, state_indices = markov_chain(
    transition_matrix, states
)

# Print the sequence of visited states
print("Sequence of Visited States:")
print(" -> ".join(state_sequence))

Sequence of Visited States:
Home -> About -> Products -> Products -> Home -> About -> Products -> Products -> Products -> Products -> Home -> Products -> Products -> Home -> About -> Home -> About -> Contact -> Home -> Products -> Home -> About -> Home -> About -> Contact -> About -> Home -> About -> Products -> Products -> Products -> About -> Home -> Products -> About -> Products -> About -> Products -> Home -> Products -> Contact -> Home -> Products -> Home -> Products -> Home -> About -> Contact -> Contact -> Products -> Contact -> Contact -> Products -> About -> Contact -> Home -> Products -> Home -> Home -> Contact -> Products -> Home -> About -> Products -> About -> About -> Products -> Products -> Home -> About -> Home -> Products -> About -> About -> Products -> Products -> Products -> Home -> About -> Products -> Home -> Products -> About -> Home -> Products -> Products -> Home -> Products -> About -> About -> Products -> About -> Home -> Products -> Products -> Home -> About

In [24]:
import pandas as pd

# Count the number of visits to each state
visit_counts = pd.Series(state_sequence).value_counts().reindex(states).fillna(0)

# Calculate the visit proportions
visit_proportions = visit_counts / len(state_sequence)

# Display the results
print("\nVisit Counts:")
print(visit_counts)

print("\nVisit Proportions:")
print(visit_proportions)


Visit Counts:
Home        26
About       26
Products    40
Contact      9
Name: count, dtype: int64

Visit Proportions:
Home        0.257426
About       0.257426
Products    0.396040
Contact     0.089109
Name: count, dtype: float64
